In [12]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from mistralai import Mistral
from dotenv import load_dotenv
import os

from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("lonely_bolivia.pdf")
pages = loader.load()

len(pages)
"""
# Charger les variables d'environnement
load_dotenv()
api_key = os.getenv('MISTRAL_API_KEY')

# Charger le texte
loader = TextLoader("raw_text_bolivie.txt")
docs = loader.load()
"""

'\n# Charger les variables d\'environnement\nload_dotenv()\napi_key = os.getenv(\'MISTRAL_API_KEY\')\n\n# Charger le texte\nloader = TextLoader("raw_text_bolivie.txt")\ndocs = loader.load()\n'

In [13]:
len(pages)

824

In [60]:
page = pages[5:250]

In [34]:
page.page_content

'Two W eeks \nThe Cordilleras & Yungas\nTrapped between the heights of the Andes and the Amazon, this fascinating\narea is a wonderland for trekkers, climbers, hikers and bikers. Getting here is\nhalf the fun. T rekkers can start from outside La Paz , traveling by foot via\nthe T akesi or Choro treks into the heart of the Y ungas. Y ou can also get into\nthe southern Y ungas on a butt-busting daylong mountain-bike ride down the\nWorld’ s Most Dangerous Road. Be sure to spend a few days at the end of\nyour descent in the pleasant Y ungas villages of Coroico or Chulumani ,\nwhich of fer plenty of day hikes, swimming and a chilled-out traveler vibe.\nFrom there, it’ s back to the capital (or via a tough circuitous jungle route) for\nclimbing and trekking in the Cordillera Real , stopping in the cool-air , soft-\nspirited Andean town of Sorata . Adventurers could take on any number of\ntreks from Sorata, good mountain-bike adventures or head up to the\nglaciated peaks of the Cordillera. Th

In [21]:
print(page.page_content[0:500])

Two W eeks 
The Cordilleras & Yungas
Trapped between the heights of the Andes and the Amazon, this fascinating
area is a wonderland for trekkers, climbers, hikers and bikers. Getting here is
half the fun. T rekkers can start from outside La Paz , traveling by foot via
the T akesi or Choro treks into the heart of the Y ungas. Y ou can also get into
the southern Y ungas on a butt-busting daylong mountain-bike ride down the
World’ s Most Dangerous Road. Be sure to spend a few days at the end of
you


In [25]:
page

Document(metadata={'source': 'lonely_bolivia.pdf', 'page': 51}, page_content='Two W eeks \nThe Cordilleras & Yungas\nTrapped between the heights of the Andes and the Amazon, this fascinating\narea is a wonderland for trekkers, climbers, hikers and bikers. Getting here is\nhalf the fun. T rekkers can start from outside La Paz , traveling by foot via\nthe T akesi or Choro treks into the heart of the Y ungas. Y ou can also get into\nthe southern Y ungas on a butt-busting daylong mountain-bike ride down the\nWorld’ s Most Dangerous Road. Be sure to spend a few days at the end of\nyour descent in the pleasant Y ungas villages of Coroico or Chulumani ,\nwhich of fer plenty of day hikes, swimming and a chilled-out traveler vibe.\nFrom there, it’ s back to the capital (or via a tough circuitous jungle route) for\nclimbing and trekking in the Cordillera Real , stopping in the cool-air , soft-\nspirited Andean town of Sorata . Adventurers could take on any number of\ntreks from Sorata, good moun

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len,
)
chunks = text_splitter.split_text(page.page_content)

In [43]:
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {len(chunk)} caractères")

Chunk 0: 947 caractères
Chunk 1: 936 caractères
Chunk 2: 580 caractères


In [61]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Créer l'instance du splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len,
)

selected_pages = pages[5:250]

# Initialiser une liste pour stocker tous les chunks avec leurs métadonnées
all_chunks_with_metadata = []

# Traiter chaque page
for page_index, page in enumerate(selected_pages):
    page_chunks = text_splitter.split_text(page.page_content)
    for chunk in page_chunks:
        all_chunks_with_metadata.append({
            "content": chunk,
            "metadata": {
                "page_number": page.metadata.get("page_number", page_index + 1),  # Utilise le numéro de page des métadonnées ou l'index + 1
                "source": page.metadata.get("source", "Unknown")
            }
        })

# Afficher le nombre de chunks créés
print(f"Nombre total de chunks créés : {len(all_chunks_with_metadata)}")

# Afficher des statistiques sur les chunks
total_pages = len(selected_pages)
chunks_per_page = [len(text_splitter.split_text(page.page_content)) for page in selected_pages]
avg_chunks_per_page = sum(chunks_per_page) / total_pages

print(f"\nStatistiques :")
print(f"Nombre total de pages traitées : {total_pages}")
print(f"Nombre moyen de chunks par page : {avg_chunks_per_page:.2f}")
print(f"Nombre minimum de chunks sur une page : {min(chunks_per_page)}")
print(f"Nombre maximum de chunks sur une page : {max(chunks_per_page)}")

# Optionnel : Afficher un exemple de chunk avec ses métadonnées
if all_chunks_with_metadata:
    print("\nExemple de chunk avec métadonnées :")
    print(all_chunks_with_metadata[100])

Nombre total de chunks créés : 469

Statistiques :
Nombre total de pages traitées : 245
Nombre moyen de chunks par page : 1.91
Nombre minimum de chunks sur une page : 0
Nombre maximum de chunks sur une page : 4

Exemple de chunk avec métadonnées :
{'content': 'OENOPHILIA\nBolivian wine doesn’ t often get the credit it deserves, but a few days spent\nsampling the goods in El V alle de la Concepción wineries near T arija will\nquickly turn you into a convert, as will the friendly folks that call this land\nof wine and honey home.\nFOOD\nChapaco  (Tarija local) cuisine is unique and inventive and, while some of\nthe dishes sound more appealing than others, you won’ t find a similar menu\nelsewhere in the country .\nClick here\nSanta Cruz & Gran Chiquitania\nFood\nHistory\nMissions\nFOOD', 'metadata': {'page_number': 73, 'source': 'lonely_bolivia.pdf'}}


In [72]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from mistralai import Mistral
from dotenv import load_dotenv
import os

# Charger les variables d'environnement
load_dotenv()
api_key = os.getenv('MISTRAL_API_KEY')

# Charger le texte
loader = TextLoader("raw_text_bolivie.txt")
docs = loader.load()


# Initialiser les embeddings Mistral
embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=api_key)

vector_store_path = "vector_store_faiss"


# Recharger le vector store à partir des fichiers sauvegardés
vector_store = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)

print("ok")

from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain


embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=api_key)

#retriever = vector_store.as_retriever()
# Define LLM
# Define prompt template
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")



"""
# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
#Cette ligne crée une chaîne de documents de type "stuff".
#La chaîne "stuff" prend tous les documents récupérés et les "bourre" (stuffs) dans un seul prompt.

retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "quelles sont les 10 meilleures activités à faire en bolivie"})
print(response["answer"])

"""

retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 20})
docs = retriever.invoke("Quelles sont les meilleures activités en Bolivie quand on est aventurier?")
print(docs)

model = ChatMistralAI(mistral_api_key=api_key)

# jaccède aux k chunk qui permettent de repondre le plus possible à la question

# Concaténer le contenu des chunks récupérés
context = "\n\n".join([doc.page_content for doc in docs])

ok
[Document(metadata={'source': 'raw_text_bolivie.txt'}, page_content='Alors, que faire en Bolivie lorsqu’il y a autant de lieux à visiter tous plus incroyables les uns que les autres ? Laissez-nous vous emmener à la découverte de notre top 5 des incontournables en Bolivie, à faire absolument si vous avez la chance de voyager en Bolivie !'), Document(metadata={'source': 'raw_text_bolivie.txt'}, page_content='Si la Selva vous permettra de découvrir la forêt, la végétation et les techniques de survie, la pampa bolivienne quant à elle est le lieux parfait pour l’observation des animaux sauvages, tels que les caïmans, les oiseaux, les serpents, les dauphins roses, les singes ou encore les capybaras. Peu importe que vous fassiez la Selva ou la pampa bolivienne, c’est dans tous les cas un incontournable absolu si vous voulez visiter la Bolivie !\n\nPour découvrir dans le détail notre guide très complet sur la Pampa d’Amazonie, vous pouvez cliquer sur le lien ci-dessous !\n\nRurrenabaque en 

In [73]:
# Concaténer le contenu des chunks récupérés
context = "\n\n".join([doc.page_content for doc in docs])

# Afficher le contenu concaténé
print("Context concaténé :\n", context)

Context concaténé :
 Alors, que faire en Bolivie lorsqu’il y a autant de lieux à visiter tous plus incroyables les uns que les autres ? Laissez-nous vous emmener à la découverte de notre top 5 des incontournables en Bolivie, à faire absolument si vous avez la chance de voyager en Bolivie !

Si la Selva vous permettra de découvrir la forêt, la végétation et les techniques de survie, la pampa bolivienne quant à elle est le lieux parfait pour l’observation des animaux sauvages, tels que les caïmans, les oiseaux, les serpents, les dauphins roses, les singes ou encore les capybaras. Peu importe que vous fassiez la Selva ou la pampa bolivienne, c’est dans tous les cas un incontournable absolu si vous voulez visiter la Bolivie !

Pour découvrir dans le détail notre guide très complet sur la Pampa d’Amazonie, vous pouvez cliquer sur le lien ci-dessous !

Rurrenabaque en Amazonie bolivienne
Visiter l’Amazonie en Bolivie depuis Rurrenabaque

Pour une journée complète, vous pouvez faire:
- Ciudad

In [74]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate

# Définir le modèle Mistral
model = ChatMistralAI(mistral_api_key=api_key)

# Définir un template pour poser la question avec le contexte concaténé
prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context:

<context>
{context}
</context>

Question: {input}
""")

# Créer le message avec le contexte concaténé et la question
question = "Quels sont les meilleurs endroits à visiter en Bolivie ?"
prompt_text = prompt.format(context=context, input=question)

# Envoyer la requête au modèle Mistral pour générer la réponse
response = model.invoke(prompt_text)
print("Réponse générée :", response)



Réponse générée : content="Based on the provided context, here are the top places to visit in Bolivie:\n\n1. La Paz: Known for its chaotic organized streets, rich culture, and colonial architecture. You can visit the Valley of las ánimas, Pico Austria, marché del alto, marché des sorcières, and show de chulletas.\n\n2. Copacabana: You can visit the sommet El Calvario here.\n\n3. Isla del sol: You can take a tour guided until the temple of the sun and can do randonnée of 6 hours around the island in autonomy.\n\n4. Sajama: You can visit the Geysers, laguna Huyana Cotta, thermes Huari.\n\n5. Torotoro: Known for Spéléologie, ciudad de itas, canyon, baignade cascade, traces dinosaures, rappel dans cascade Chiflon.\n\n6. Sucre: Known for its colonial architecture, you can do food tour, horse riding here.\n\n7. Amazonie: You can visit the pampa or selva bolivienne for wildlife observation.\n\n8. Salar d'Uyuni and the sud Lipez: Known for providing one of the most extraordinary experiences in